# 在 Jupyter Notebook 執行 asyncio 程式
Jupyter Notebook 的 event loop 已經啟動，直接使用 `await` 會出現 `'await' outside function` 錯誤。  
可使用 `nest_asyncio` 套件解決此問題。

In [8]:
# 安裝 nest_asyncio 套件
!pip install nest_asyncio

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
import asyncio
async def say_hello(name, delay):
    print(f"{name}開始執行")
    await asyncio.sleep(delay) #模擬I/O等待
    print(f"{name}完成,耗時{delay}秒")

async def main():
    # 同時執行多個任務
    await asyncio.gather(say_hello("任務1",2), say_hello("任務2", 1))

await main()

任務1開始執行
任務2開始執行
任務2完成,耗時1秒
任務1完成,耗時2秒


In [1]:
import asyncio
async def fetch_data(url, delay):
    print(f'開始抓取{url}')
    await asyncio.sleep(delay)
    print(f"完成抓取{url}")
    return f"資料來自{url}"

async def main():
    urls = [('網站1',2), ('網站2', 1), ('網站3', 3)]
    tasks = [fetch_data(url, delay) for url, delay in urls]
    result = await asyncio.gather(*tasks)
    print('所有資料:',result)

await main()

開始抓取網站1
開始抓取網站2
開始抓取網站3
完成抓取網站2
完成抓取網站1
完成抓取網站3
所有資料: ['資料來自網站1', '資料來自網站2', '資料來自網站3']


In [3]:
import time 

def task(id, delay):
    print(f'開始任務{id}')
    time.sleep(delay)
    print(f'任務{id}完成')

start = time.time()
task(1, 1)
task(2, 2)
print(f'同步總耗時:{time.time() - start:.2f}秒')

開始任務1
任務1完成
開始任務2
任務2完成
同步總耗時:3.01秒


In [5]:
import asyncio
async def task(id, delay):
    print(f'開始任務{id}')
    await asyncio.sleep(delay)
    print(f'任務{id}完成')

start = time.time()
async def main():
    tasks = [task(1,1), task(2,2)]
    await asyncio.gather(*tasks)
await main()
print(f'同步總耗時:{time.time() - start:.2f}秒')

開始任務1
開始任務2
任務1完成
任務2完成
同步總耗時:2.00秒


In [6]:
import asyncio
async def task(id, delay):
    print(f'開始任務{id}')
    await asyncio.sleep(delay)
    print(f'任務{id}完成')

start = time.time()
async def main():
    await task(1,1)
    await task(2,2)
    
await main()
print(f'同步總耗時:{time.time() - start:.2f}秒')

開始任務1
任務1完成
開始任務2
任務2完成
同步總耗時:3.00秒


In [7]:
import asyncio
async def task(id, delay):
    print(f'開始任務{id}')
    await asyncio.sleep(delay)
    print(f'任務{id}完成')

start = time.time()
async def main():
    task1 = asyncio.create_task(task(1,1))
    task2 = asyncio.create_task(task(2,2))
    await task1
    await task2
   
    
await main()
print(f'同步總耗時:{time.time() - start:.2f}秒')

開始任務1
開始任務2
任務1完成
任務2完成
同步總耗時:2.00秒


In [2]:
import asyncio
import aiohttp

async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f"抓取{url}完成")
            return await response.text()

async def main():
    urls = ["https://example.com", "https://python.org", "https://github.com"]
    tasks = [fetch_url(url) for url in urls]
    results = await asyncio.gather(*tasks)
    for url, content in zip(urls, results):
        print(f"{url} 的內容：\n{content[:200]}...\n")  # 只顯示前200字，避免太長

await main()

抓取https://github.com完成
抓取https://python.org完成
抓取https://example.com完成
https://example.com 的內容：
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" conten...

https://python.org 的內容：
<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <h...

https://github.com 的內容：








<!DOCTYPE html>
<html
  lang="en"
  data-color-mode="dark" data-dark-theme="dark"
  data-color-mode="light" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system"...

抓取https://python.org完成
抓取https://example.com完成
https://example.com 的內容：
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
  

In [ ]:
import asyncio
import aiohttp

async def fetch_url(url):
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url,timeout=2) as response:
                return await response.text()
    except Exception as e:
        print(f'抓取{url}失數:{e}')
        return None

async def main():
    urls =  ["https://example.com", "https://invalid-url", "https://python.org"]
    results = await asyncio.gather(*[fetch_url(url) for url in urls], return_exceptions=True)
    for url, content in zip(urls, results):
        if content is None:
            print(f"{url} 發生錯誤\n")
        else:
            print(f"{url} 的內容：\n{content[:200]}...\n")  # 只顯示前200字，避免太長

await main()

抓取https://invalid-url失數:Cannot connect to host invalid-url:443 ssl:default [nodename nor servname provided, or not known]
https://example.com 的內容：
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" conten...



TypeError: 'NoneType' object is not subscriptable